In [2]:
import os 
import time 
import json,math,sys,re
import pandas as pd 
from tqdm import tqdm  

In [3]:
folder = "../.."
from dotenv import load_dotenv
env_path = f'{folder}/deepseek.env'
load_dotenv(dotenv_path=env_path)
DEEPSEEK_KEY = os.getenv('DEEPSEEK_KEY')
from openai import OpenAI
client = OpenAI(api_key=DEEPSEEK_KEY, base_url="https://api.deepseek.com")

In [5]:
def clean_text(s): 
    s = re.sub(r"\<\/i\>|\<NOTE\>|NONLATINALPHABET|\<i\>|\d+\^PAGE[S]*\^MISSING","",s)
    s = re.sub(r"\s+"," ",s)
    s = s.strip(" ")
    return s 
all_o,all_c = [],[]
main_repo = "/Users/amycweng/SERMONS_APP/db/data/"
c_files = sorted(k for k in os.listdir(f'{main_repo}/CITATIONS') if ".csv" in k)
prior_era = sorted(c_files)[0].split("_citations.csv")[0].split("_")[0]
for file in tqdm(sorted(c_files)): 
    era, prefix = file.split("_citations.csv")[0].split("_")
    if prior_era != era: 
        all_c_df = pd.DataFrame(all_c)
        all_c_df.to_csv(f"../assets/CITATIONS_WITH_CONTEXTS/{prior_era}_citations.csv",index=False)
        all_c = []
        prior_era = era 
    # print(era,prefix)
    data = pd.read_csv(f"{main_repo}/CITATIONS/{file}",header=None)
    data = data.groupby([0,1,2], as_index=False).agg({
    3: lambda x: '; '.join(x.dropna().astype(str).unique()),
    4: lambda x: '; '.join(x.dropna().astype(str).unique()),
    5: lambda x: '; '.join(x.dropna().astype(str).unique()),
    6: lambda x: '; '.join(x.dropna().astype(str).unique())
    })
    data = data.to_dict(orient='records')
    data = {(d[0],d[1],d[2]):d for d in data}
    text = pd.read_csv(f"{main_repo}/{era}/{prefix}_body.csv",header=None).to_dict(orient='records')
    text = {(d[0],d[1],"In-Text"):clean_text(d[6]) for d in text}
    for key, t in text.items(): 
        if key in data: 
            data[key]['text'] = t 
    if f"{prefix}_margin.csv" in os.listdir(f"{main_repo}/{era}"): 
        text = pd.read_csv(f"{main_repo}/{era}/{prefix}_margin.csv",header=None).to_dict(orient='records')
        text = {(d[0],d[1],"Note " + str(d[2])):clean_text(d[3]) for d in text}
        for key, t in text.items(): 
            if key in data: 
                data[key]['text'] = t
    for d in data.values(): 
        if len(d[5]) == 0: 
            all_c.append({'Phrase': d[6], 'Citations':d[4], "Outliers":d[5],'Text': d['text']})
        else: 
            all_o.append({'Phrase': d[6], 'Citations':d[4], "Outliers":d[5],'Text': d['text']})
all_c_df = pd.DataFrame(all_c)
all_c_df.to_csv(f"../assets/CITATIONS_WITH_CONTEXTS/{prior_era}_citations.csv",index=False)
all_o_df = pd.DataFrame(all_o)
all_o_df.to_csv("../assets/CITATIONS_WITH_CONTEXTS/citation_outliers.csv",index=False)
all_o_df

  4%|▍         | 3/76 [01:04<30:08, 24.77s/it]/var/folders/3_/ylrg8wdj20l755p4921q0wg80000gp/T/ipykernel_56805/1195936537.py:27: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  text = pd.read_csv(f"{main_repo}/{era}/{prefix}_body.csv",header=None).to_dict(orient='records')
 36%|███▌      | 27/76 [06:55<06:44,  8.25s/it]/var/folders/3_/ylrg8wdj20l755p4921q0wg80000gp/T/ipykernel_56805/1195936537.py:27: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  text = pd.read_csv(f"{main_repo}/{era}/{prefix}_body.csv",header=None).to_dict(orient='records')
 84%|████████▍ | 64/76 [13:52<02:18, 11.57s/it]/var/folders/3_/ylrg8wdj20l755p4921q0wg80000gp/T/ipykernel_56805/1195936537.py:27: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  text = pd.read_csv(f"{main_repo}/{era}/{prefix}_body.csv",header=None).to_dict(orient='records')
100%|█████████

,Phrase,Citations,Outliers,Text
0,Lev. 26. 4•.,Leviticus 26,['Leviticus 26.4•'],Hence the Lord promises Covenant good to those...
1,Psal. 143.; Psal. 119. 13•.,Psalms 143; Psalms 119,['Psalms 119.13•'],Therefore (saith David) I said I would take he...
2,Joh. 10. 1•.,John 10,['John 10.1•'],Joh. 10. 1•.
3,Rom 3. ◊,Romans 3,['Romans 3.◊'],"Tea and particular provincial Councels, though..."
4,Act. 11. 22. &; Gal. 1. 1•.,Acts 11.22; Galatians 1,['Galatians 1.1•'],Barn•bas sent by the Church of Jerusalem to An...
...,...,...,...,...
5543,Mat. iii.; Mark. i. •; Luke. iii. c.; Iohn. i.,Matthew 3; Mark 1; Luke 3; John 1,['Mark 1 •'],Mat. iii. b Mark. i. • Luke. iii. c. Iohn. i. d.
5544,1. Cor. iii. •,1 Corinthians 3,['1 Corinthians 3 •'],"1. Cor. iii. • Iohn. vi, b Actum. ii. •"
5545,Roma. x •,Romans 10,['Romans 10 •'],Roma. x •
5546,Ioh. xiii. •,John 13,['John 13 •'],Ioh. xiii. •


In [ ]:
all_o_df = pd.read_csv("../assets/CITATIONS_WITH_CONTEXTS/citation_outliers.csv")
all_o_df['Outliers'] = all_o_df['Outliers'].str.strip("']").str.strip("'[") 
all_o_df = all_o_df.reset_index()
all_o_df = all_o_df.rename(columns={'index': 'Index'})
all_o_df = all_o_df.to_dict(orient='records')
output = []
def get_book(t):
   return " ".join(t.split(".")[0].split()[:-1]) 
new_idx = 125
for entry in all_o_df: 
    index = entry['Index']
    if index < 90: continue

    c = entry['Citations'].split("; ")
    o = entry['Outliers'].split("; ")
    p = entry['Phrase'].split("; ")
    entry['Index'] = new_idx 
    new_idx += 1  
    if len(c) == 1: 
        entry['Citations'] = entry['Outliers']
        output.append(entry)
    else: 
        for idx in range(len(p)):
            new_entry = entry.copy()
            if idx > 0: 
                new_entry['Text'] = ''
            new_entry['Phrase'] = re.sub(r" \&$| \&c[,.]$",'',p[idx]).strip()
            new_c = []
            if len(c) == 0: 
                if len(new_c) > 0: 
                    new_entry['Text'] = ''
                    new_entry['Phrase'] = ''
                new_entry['Citations'] = entry['Outliers']
            else: 
                match_book = get_book(c[0])
                remaining = []
                for item in c: 
                    if match_book == get_book(item): 
                        found = False 
                        remaining_o = []
                        for o_item in o: 
                            if re.search(item,o_item): 
                                new_c.append(o_item)
                                found = True 
                            else: 
                                remaining_o.append(o_item)
                        o = remaining_o
                        if not found: 
                            new_c.append(item)
                    else: 
                        remaining.append(item)
                c = remaining
            new_entry['Citations'] = '; '.join(new_c)
            output.append(new_entry)
output = pd.DataFrame(output)
output = output[['Text','Index','Phrase','Citations']]
output.to_csv("../assets/CITATIONS_WITH_CONTEXTS/citation_outliers_reformatted.csv",index=False)

In [6]:
pattern = r'^(Kings|Chronicles|Samuel|Corinthians|Timothy|Thessalonians|Peter|Maccabees|Esdras|Paralipomenon)'
for file in os.listdir('../assets/CITATIONS_WITH_CONTEXTS'): 
    if file.split("_")[0] not in ['pre-Elizabeth','JamesII']: 
        continue 
    data = pd.read_csv(f"../assets/CITATIONS_WITH_CONTEXTS/{file}")
    data = data[data['Outliers'].isna()]
    data = data[
                data['Citations'].str.contains('Epistle|Verse|Ibidem') |
                data['Phrase'].str[0].str.islower() | 
                data['Phrase'].str[0].str.islower() | 
                data['Phrase'].str.contains(r'^[\d\s.,]*\b[A-Za-z]{1}\b[\s:.,]', regex=True, na=False) |
                data['Citations'].str.contains(pattern, case=False, na=False, regex=True)
                ]
    data['Phrase'] = data['Phrase'].str.strip(" &")
data = data.reset_index()
data = data.rename(columns={'index': 'Index'})
data = data.to_dict(orient='records')
pd.DataFrame(data)

/var/folders/3_/ylrg8wdj20l755p4921q0wg80000gp/T/ipykernel_56805/3613374772.py:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data['Citations'].str.contains(pattern, case=False, na=False, regex=True)
/var/folders/3_/ylrg8wdj20l755p4921q0wg80000gp/T/ipykernel_56805/3613374772.py:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data['Citations'].str.contains(pattern, case=False, na=False, regex=True)


,Index,Phrase,Citations,Outliers,Text
0,19,Peter. 2.13.,Peter 2.13,NaN,1. St Peter. 2.13.
1,27,verse. 20.,Verse 20,NaN,verse. 20.
2,180,2 K. 13.,2 Kings 13,NaN,2 K. 13.
3,382,ho. 10.,Hosea 10,NaN,Bas. ho. 10. in exam.
4,386,ho. 2.,Hosea 2,NaN,Orig. expos. 2. Can. ho. 2.
...,...,...,...,...,...
835,19060,col. 1.,Colossians 1,NaN,Benefits arising to the Common wealth from Mon...
836,19061,col. 1.157.; col. 2.105.,Colossians 1.157; Colossians 2.105,NaN,V. Sr. Wil. Dugdale Hist. Warwickshire pag. 14...
837,19131,Ep. 3.,Epistle 3,NaN,S. Pacian. Ep. 3. ad Synphr.
838,19170,"Ibid v. 8, 9.; Exod. chap. 8. v. 9.",Ibidem 8; Ibidem 9; Exodus 8.9,NaN,"For his Wrath shall come suddenly, and in the ..."


In [7]:
output = []
def get_book(t):
   return " ".join(t.split(".")[0].split()[:-1]) 
new_idx = 5588
for entry in data: 
    index = entry['Index']
    c = str(entry['Citations']).split("; ")
    p = entry['Phrase'].split("; ")
    entry['Index'] = new_idx 
    new_idx += 1  
    for idx in range(len(p)):
        new_entry = entry.copy()
        if idx > 0: 
            new_entry['Text'] = ''
        new_entry['Phrase'] = re.sub(r" \&$| \&c[,.]$",'',p[idx]).strip()
        new_c = []
        if len(c) == 0: 
            if len(new_c) > 0: 
                new_entry['Text'] = ''
                new_entry['Phrase'] = ''
            new_entry['Citations'] = entry['Outliers']
        else: 
            match_book = get_book(c[0])
            remaining = []
            for item in c: 
                if match_book == get_book(item): 
                    new_c.append(item)
                else: 
                    remaining.append(item)
            c = remaining
        new_entry['Citations'] = '; '.join(new_c)
        output.append(new_entry)
output = pd.DataFrame(output)
output = output[['Text','Index','Phrase','Citations']]
output

,Text,Index,Phrase,Citations
0,1. St Peter. 2.13.,5588,Peter. 2.13.,Peter 2.13
1,verse. 20.,5589,verse. 20.,Verse 20
2,2 K. 13.,5590,2 K. 13.,2 Kings 13
3,Bas. ho. 10. in exam.,5591,ho. 10.,Hosea 10
4,Orig. expos. 2. Can. ho. 2.,5592,ho. 2.,Hosea 2
...,...,...,...,...
993,,6424,col. 2.105.,
994,S. Pacian. Ep. 3. ad Synphr.,6425,Ep. 3.,Epistle 3
995,"For his Wrath shall come suddenly, and in the ...",6426,"Ibid v. 8, 9.",Ibidem 8; Ibidem 9
996,,6426,Exod. chap. 8. v. 9.,Exodus 8.9


In [8]:
output.to_csv("../assets/CITATIONS_WITH_CONTEXTS/citations_more_outliers_reformatted.csv",index=False)

## LLM Testing 

In [7]:
responses = {}
model = "V3" 
fname = "citations"
with open(f"{folder}/QP/predictions/{fname}_DS{model}.json",'r') as f: 
    responses = json.load(f) 
list(responses.items())[0]

('0', '1. True; False\n2. Matthew 19; Matthew 21.3')

In [ ]:
prompt = """You are given phrases of Early Modern English that may contain Biblical citations of various formats. 
            You are also given the possible parsings of the citation and the textual context of the phrase.  
            As an Early Modern English and Latin expert, you determine whether the parsings are actual citations in their texts. 
            Ignore the unrelated citations, if any, in the context. M
            Return outputs in the following format: 
            1. True/False for each parsed citation separated by semicolons
            2. How you would accurately parse and standardize that phrase into individual citations if it contains any. GIVE NO EXPLANATIONS. 

            For example, you might get the following input:
            Phrase: 'Gal. 5.20. & 21.'
            Possible Citations: 'Galatians 5.20';'Galatians 5.21'
            Context: 'avoyding of and suppressing, all wrath, envy, malice, revenge and the like sinful passions that have the stamp of Hell upon them, ( James 3.15.) which come thence and lead thither, Gal. 5.20. and 21.'

            You should return the following output: 
            1. True; True
            2. Galatians 5.20; Galatians 5.21 

            Another example is as follows. This example has no citations because the "3." following "Isaiah" is a number in a list rather than a chapter citation. 
            Phrase: 'Isaiah. 3. •olinesse of'
            Possible Citations: 'Isaiah 3'; 'Isaiah 3.•olinesse of'
            Context: 'It is a fine saying, that there are three things necessary for a Preacher. 1. Knowledge, noted in the book sent to Ezekiel, 2 Eloquence, in the hote coal that purged Isaiah. 3. •olinesse of life, in the hand sent to Jeremiah wanting Utterance, he is an Insant;'

            Output: 
            1. False; False
            2. None

            Another example that has one true citation but another false one.
            Phrase: 'Matth. 7.44, & c &'
            Possible Citations: 'Matthew 7.44', 'Matthew 7.100'
            Context: "By no means rendering railing for railing, or evil for evil, but overcoming evil with good, Oh that our blessed Saviour's known precept, was but as known in the practice, Matth. 7.44, & c! and that his copy was but writ after by us;"

            Output: 
            1. True; False
            2. Matthew 7.44

        """

In [ ]:
responses = {}
model = "V3" 
fname = "citations"

In [ ]:
from tqdm import tqdm 

for idx, phrase in enumerate(tqdm(all_c)):
    if idx in responses: continue 

    citation, outlier, text = all_c[phrase]
    outlier = re.findall(r"\'(.*?)\'",outlier)
    citation = citation + "; " + "; ".join(outlier)
    response = client.chat.completions.create(
        model='deepseek-chat',
        # max_tokens = , 
        messages=[ 
            {"role": "user",  
             "content": f"{prompt}\n\nPhrase: {phrase}\nPossible Citations: {citation}\nContext: {text}\n\nOutput:"
            }
        ],
        stream=False
    )
    
    responses[idx] = response.choices[0].message.content
    if (idx+1) % 500 == 0: 
        with open(f"{folder}/QP/predictions/{fname}_DS{model}.json",'w') as f: 
            json.dump(responses,f) 

with open(f"{folder}/QP/predictions/{fname}_DS{model}.json",'w') as f: 
    json.dump(responses,f) 
    # 100%|██████████| 4380/4380 [4:56:13<00:00,  4.06s/it] 
    # 100%|██████████| 3291/3291 [3:09:16<00:00,  3.45s/it]

In [76]:
with open(f"{folder}/QP/predictions/{fname}_DS{model}.json",'w') as f: 
    json.dump(responses,f) 

In [ ]:
list(responses.values())[:10]

In [ ]:
list(responses.values())[:10]

In [ ]:
list(all_c.items())[:7]